In [85]:
import sqlite3
import pandas as pd
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

### 
    Была проведена работа с базой данных,написан запрос для создания таблицы с признаками для анализа и предсказания(создания модели):SELECT o.order_id, o.order_dt, o.payment_method, o.customer_id, po.product_id,po.amount,
    p.product_group_name , p.product_name,p.Price,c.address,
    a.start_dttm,a.finish_dttm,a.device_name,datediff(a.start_dttm,a.finish_dttm) as delivery_time
    FROM orders AS o
    JOIN orders_product AS po ON o.order_id = po.order_id
    JOIN products AS p ON po.product_id = p.product_id
    JOIN customers AS c ON o.customer_id = c.customer_id
    JOIN app_sess AS a ON c.customer_id = a.customer_id;
    После чего были загруженны файлы с таблицами из базы данных и на основе запроса приступим к созданию витрины данных


In [86]:
app = pd.read_csv('app_sess.csv')

In [87]:
prod = pd.read_csv('products.csv')

In [88]:
order = pd.read_csv('orders.csv')

In [89]:
orprod = pd.read_csv('orders_product.csv')

In [90]:
cust = pd.read_csv('customers.csv')

In [91]:
order

,order_id,customer_id,order_dt,payment_method
0,99983,4951223,07.12.2021,offline
1,99969,1806908,29.07.2022,offline
2,99868,2576384,27.09.2021,online
3,99757,816871,02.04.2021,offline
4,99710,7864358,26.01.2020,online
...,...,...,...,...
2945,112,9565866,11.04.2023,online
2946,109,4494818,13.09.2022,offline
2947,105,8042250,03.11.2021,online
2948,97,6080668,23.04.2021,online


In [215]:
delivery = pd.merge(order,orprod,left_on='order_id',right_on='order_id',how='inner')

In [216]:
delivery = pd.merge(delivery,prod,left_on='product_id',right_on='product_id',how='inner')

In [217]:
delivery = pd.merge(delivery,cust,left_on='customer_id',right_on='customer_id',how='inner')

In [218]:
delivery = pd.merge(delivery,app,left_on='customer_id',right_on='customer_id',how='inner')

In [219]:
delivery

,order_id,customer_id,order_dt,payment_method,product_id,amount,product_group_name,product_name,Price,last_name,first_name,middle_name,phone_number,birth_dt,address,session_id,start_dttm,finish_dttm,device_name
0,99983,4951223,07.12.2021,offline,3,24,Одежда,Юбка,10000,Андреев,Михаил,Глебович,800533.0,22.04.1999,Уссурийск,380,06.12.21 7:06,07.12.21 8:38,MacOS
1,99983,4951223,07.12.2021,offline,3,24,Одежда,Юбка,10000,Андреев,Михаил,Глебович,800533.0,22.04.1999,Уссурийск,716,06.12.21 15:38,08.12.21 2:40,MacOS
2,99983,4951223,07.12.2021,offline,3,24,Одежда,Юбка,10000,Андреев,Михаил,Глебович,800533.0,22.04.1999,Уссурийск,1546,06.12.21 20:51,08.12.21 0:18,iOS
3,99983,4951223,07.12.2021,offline,3,24,Одежда,Юбка,10000,Андреев,Михаил,Глебович,800533.0,22.04.1999,Уссурийск,1798,06.12.21 16:21,07.12.21 18:33,MacOS
4,99983,4951223,07.12.2021,offline,3,24,Одежда,Юбка,10000,Андреев,Михаил,Глебович,800533.0,22.04.1999,Уссурийск,2150,06.12.21 1:59,06.12.21 2:05,Android
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64077,10503,6739793,06.03.2023,offline,1,43,Одежда,Носки,500,Иванова,Кира,Ярославовна,862044.0,14.11.1971,Ачинск,3333,19.04.23 14:07,19.04.23 15:19,Android
64078,10503,6739793,06.03.2023,offline,1,43,Одежда,Носки,500,Иванова,Кира,Ярославовна,862044.0,14.11.1971,Ачинск,3590,19.04.23 13:48,20.04.23 10:04,MacOS
64079,10503,6739793,06.03.2023,offline,1,43,Одежда,Носки,500,Иванова,Кира,Ярославовна,862044.0,14.11.1971,Ачинск,4417,19.04.23 4:38,19.04.23 14:13,Ubuntu
64080,10503,6739793,06.03.2023,offline,1,43,Одежда,Носки,500,Иванова,Кира,Ярославовна,862044.0,14.11.1971,Ачинск,4678,19.04.23 7:44,20.04.23 11:03,Android


####
    Удаление ненужных атрибутов из витрины данных
    

In [220]:
delivery = delivery.drop(['order_dt','last_name','first_name','middle_name','birth_dt','session_id','order_id',
                          'customer_id','product_id'],axis=1)

In [221]:
from datetime import datetime, timedelta

In [222]:
delivery['start_dttm']=pd.to_datetime(delivery['start_dttm'])
delivery['finish_dttm']=pd.to_datetime(delivery['finish_dttm'])

In [223]:
delivery['delivery_time'] = delivery['start_dttm'] - delivery['finish_dttm']

In [224]:
delivery['delivery_time'] = delivery['delivery_time'].dt.floor("D")

In [225]:
delivery

,payment_method,amount,product_group_name,product_name,Price,phone_number,address,start_dttm,finish_dttm,device_name,delivery_time
0,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,2021-06-12 07:06:00,2021-07-12 08:38:00,MacOS,-31 days
1,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,2021-06-12 15:38:00,2021-08-12 02:40:00,MacOS,-61 days
2,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,2021-06-12 20:51:00,2021-08-12 00:18:00,iOS,-61 days
3,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,2021-06-12 16:21:00,2021-07-12 18:33:00,MacOS,-31 days
4,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,2021-06-12 01:59:00,2021-06-12 02:05:00,Android,-1 days
...,...,...,...,...,...,...,...,...,...,...,...
64077,offline,43,Одежда,Носки,500,862044.0,Ачинск,2023-04-19 14:07:00,2023-04-19 15:19:00,Android,-1 days
64078,offline,43,Одежда,Носки,500,862044.0,Ачинск,2023-04-19 13:48:00,2023-04-20 10:04:00,MacOS,-1 days
64079,offline,43,Одежда,Носки,500,862044.0,Ачинск,2023-04-19 04:38:00,2023-04-19 14:13:00,Ubuntu,-1 days
64080,offline,43,Одежда,Носки,500,862044.0,Ачинск,2023-04-19 07:44:00,2023-04-20 11:03:00,Android,-2 days


In [226]:
delivery = delivery.drop(['start_dttm','finish_dttm'],axis=1)

In [227]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split 

In [228]:
delivery.isnull().sum()

payment_method        0
amount                0
product_group_name    0
product_name          0
Price                 0
phone_number          0
address               0
device_name           0
delivery_time         0
dtype: int64

In [229]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64082 entries, 0 to 64081
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   payment_method      64082 non-null  object         
 1   amount              64082 non-null  int64          
 2   product_group_name  64082 non-null  object         
 3   product_name        64082 non-null  object         
 4   Price               64082 non-null  int64          
 5   phone_number        64082 non-null  float64        
 6   address             64082 non-null  object         
 7   device_name         64082 non-null  object         
 8   delivery_time       64082 non-null  timedelta64[ns]
dtypes: float64(1), int64(2), object(5), timedelta64[ns](1)
memory usage: 4.4+ MB


In [230]:
delivery['delivery_time'] = pd.to_numeric(delivery['delivery_time'])

In [231]:
delivery

,payment_method,amount,product_group_name,product_name,Price,phone_number,address,device_name,delivery_time
0,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-2678400000000000
1,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-5270400000000000
2,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,iOS,-5270400000000000
3,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-2678400000000000
4,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,Android,-86400000000000
...,...,...,...,...,...,...,...,...,...
64077,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-86400000000000
64078,offline,43,Одежда,Носки,500,862044.0,Ачинск,MacOS,-86400000000000
64079,offline,43,Одежда,Носки,500,862044.0,Ачинск,Ubuntu,-86400000000000
64080,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-172800000000000


In [232]:
delivery['delivery_time'] = delivery['delivery_time'].apply(lambda x:x/86400)

In [233]:
delivery

,payment_method,amount,product_group_name,product_name,Price,phone_number,address,device_name,delivery_time
0,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-3.100000e+10
1,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-6.100000e+10
2,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,iOS,-6.100000e+10
3,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-3.100000e+10
4,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,Android,-1.000000e+09
...,...,...,...,...,...,...,...,...,...
64077,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-1.000000e+09
64078,offline,43,Одежда,Носки,500,862044.0,Ачинск,MacOS,-1.000000e+09
64079,offline,43,Одежда,Носки,500,862044.0,Ачинск,Ubuntu,-1.000000e+09
64080,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-2.000000e+09


In [234]:
delivery['delivery_time'] = delivery['delivery_time'].apply(lambda x:round(x))

In [235]:
delivery

,payment_method,amount,product_group_name,product_name,Price,phone_number,address,device_name,delivery_time
0,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-31000000000
1,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-61000000000
2,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,iOS,-61000000000
3,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-31000000000
4,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,Android,-1000000000
...,...,...,...,...,...,...,...,...,...
64077,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-1000000000
64078,offline,43,Одежда,Носки,500,862044.0,Ачинск,MacOS,-1000000000
64079,offline,43,Одежда,Носки,500,862044.0,Ачинск,Ubuntu,-1000000000
64080,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-2000000000


In [236]:
delivery['delivery_time'] = delivery['delivery_time'].apply(lambda x:x/1000000000)

In [237]:
delivery

,payment_method,amount,product_group_name,product_name,Price,phone_number,address,device_name,delivery_time
0,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-31.0
1,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-61.0
2,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,iOS,-61.0
3,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,MacOS,-31.0
4,offline,24,Одежда,Юбка,10000,800533.0,Уссурийск,Android,-1.0
...,...,...,...,...,...,...,...,...,...
64077,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-1.0
64078,offline,43,Одежда,Носки,500,862044.0,Ачинск,MacOS,-1.0
64079,offline,43,Одежда,Носки,500,862044.0,Ачинск,Ubuntu,-1.0
64080,offline,43,Одежда,Носки,500,862044.0,Ачинск,Android,-2.0


In [260]:
delivery['delivery_time'] = delivery['delivery_time'].astype("Int64")

In [261]:
delivery

,amount,Price,phone_number,delivery_time,payment_method_offline,payment_method_online,product_group_name_Напитки,product_group_name_Одежда,product_group_name_Продукты,product_group_name_Часы,...,address_Южно-Сахалинск,address_Юрга,address_Якутск,address_Ялта,address_Ярославль,device_name_Android,device_name_MacOS,device_name_Ubuntu,device_name_Windows,device_name_iOS
0,-0.098235,0.063419,-1.719530,-31,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.098235,0.063419,-1.719530,-61,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.098235,0.063419,-1.719530,-61,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.098235,0.063419,-1.719530,-31,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.098235,0.063419,-1.719530,-1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64077,1.204234,-0.353283,0.435245,-1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
64078,1.204234,-0.353283,0.435245,-1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
64079,1.204234,-0.353283,0.435245,-1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
64080,1.204234,-0.353283,0.435245,-2,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [238]:
std_scaler = StandardScaler()

In [239]:
number = delivery[['amount','Price','phone_number']]

In [240]:
std_scaler.fit(number)

StandardScaler()

In [241]:
std_scaler_data = std_scaler.transform(number)

In [242]:
delivery[['amount','Price','phone_number']] = std_scaler_data
delivery

,payment_method,amount,product_group_name,product_name,Price,phone_number,address,device_name,delivery_time
0,offline,-0.098235,Одежда,Юбка,0.063419,-1.719530,Уссурийск,MacOS,-31.0
1,offline,-0.098235,Одежда,Юбка,0.063419,-1.719530,Уссурийск,MacOS,-61.0
2,offline,-0.098235,Одежда,Юбка,0.063419,-1.719530,Уссурийск,iOS,-61.0
3,offline,-0.098235,Одежда,Юбка,0.063419,-1.719530,Уссурийск,MacOS,-31.0
4,offline,-0.098235,Одежда,Юбка,0.063419,-1.719530,Уссурийск,Android,-1.0
...,...,...,...,...,...,...,...,...,...
64077,offline,1.204234,Одежда,Носки,-0.353283,0.435245,Ачинск,Android,-1.0
64078,offline,1.204234,Одежда,Носки,-0.353283,0.435245,Ачинск,MacOS,-1.0
64079,offline,1.204234,Одежда,Носки,-0.353283,0.435245,Ачинск,Ubuntu,-1.0
64080,offline,1.204234,Одежда,Носки,-0.353283,0.435245,Ачинск,Android,-2.0


In [243]:
categorie = delivery[['payment_method','product_group_name','product_name','address','device_name']]

In [244]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(categorie)

OneHotEncoder(sparse_output=False)

In [245]:
ohe1 = ohe.transform(categorie)
ohe1

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [246]:
ohe.get_feature_names_out()

array(['payment_method_offline', 'payment_method_online',
       'product_group_name_Напитки', 'product_group_name_Одежда',
       'product_group_name_Продукты', 'product_group_name_Часы',
       'product_name_Вода', 'product_name_Злой кола',
       'product_name_Круассан', 'product_name_Молоко',
       'product_name_Носки', 'product_name_Огурец',
       'product_name_Рубашка', 'product_name_Сок', 'product_name_Хлеб',
       'product_name_Часы напольные', 'product_name_Часы наручные',
       'product_name_Часы настенные', 'product_name_Часы с кукушкой',
       'product_name_Штаны', 'product_name_Юбка', 'address_Абакан',
       'address_Азов', 'address_Александров', 'address_Алексин',
       'address_Альметьевск', 'address_Анапа', 'address_Ангарск',
       'address_Анжеро-Судженск', 'address_Апатиты', 'address_Арзамас',
       'address_Армавир', 'address_Арсеньев', 'address_Артем',
       'address_Архангельск', 'address_Асбест', 'address_Астрахань',
       'address_Ачинск', 'address_Бал

In [247]:
delivery[ohe.get_feature_names_out()] = ohe1
delivery.head()

,payment_method,amount,product_group_name,product_name,Price,phone_number,address,device_name,delivery_time,payment_method_offline,...,address_Южно-Сахалинск,address_Юрга,address_Якутск,address_Ялта,address_Ярославль,device_name_Android,device_name_MacOS,device_name_Ubuntu,device_name_Windows,device_name_iOS
0,offline,-0.098235,Одежда,Юбка,0.063419,-1.71953,Уссурийск,MacOS,-31.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,offline,-0.098235,Одежда,Юбка,0.063419,-1.71953,Уссурийск,MacOS,-61.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,offline,-0.098235,Одежда,Юбка,0.063419,-1.71953,Уссурийск,iOS,-61.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,offline,-0.098235,Одежда,Юбка,0.063419,-1.71953,Уссурийск,MacOS,-31.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,offline,-0.098235,Одежда,Юбка,0.063419,-1.71953,Уссурийск,Android,-1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [248]:
delivery = delivery.drop(['payment_method','product_group_name','product_name','address','device_name'],axis=1)

In [249]:
delivery

,amount,Price,phone_number,delivery_time,payment_method_offline,payment_method_online,product_group_name_Напитки,product_group_name_Одежда,product_group_name_Продукты,product_group_name_Часы,...,address_Южно-Сахалинск,address_Юрга,address_Якутск,address_Ялта,address_Ярославль,device_name_Android,device_name_MacOS,device_name_Ubuntu,device_name_Windows,device_name_iOS
0,-0.098235,0.063419,-1.719530,-31.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.098235,0.063419,-1.719530,-61.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.098235,0.063419,-1.719530,-61.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.098235,0.063419,-1.719530,-31.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.098235,0.063419,-1.719530,-1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64077,1.204234,-0.353283,0.435245,-1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
64078,1.204234,-0.353283,0.435245,-1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
64079,1.204234,-0.353283,0.435245,-1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
64080,1.204234,-0.353283,0.435245,-2.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [250]:
df_prepared = delivery.copy()

x = df_prepared.drop(['delivery_time'], axis=1)
y = df_prepared['delivery_time']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

In [251]:
lg = LogisticRegression(max_iter = 300, C=3)
lg.fit(x_train,y_train)
logpred = lg.predict(x_test)
accuracy_score(y_test,logpred)

0.561502750360863

In [253]:
base_rf2 = RandomForestClassifier()
base_rf2.fit(x_train, y_train)
preds2 = base_rf2.predict(x_test)
accuracy_score(y_test,preds2)

0.5481995864705653

In [254]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
param_grid = {
   'n_estimators': list(range(100, 501, 100)),
   'max_features': ['sqrt', 'log2'],
   'min_samples_leaf': list(range(1, 10))
}
randomized_search_rf = RandomizedSearchCV(
   estimator=rf,
   param_distributions=param_grid,
   n_iter=20,
   scoring='accuracy',
   verbose=1,
   n_jobs=-1
)

randomized_search_rf.fit(x_train, y_train)
best_params = randomized_search_rf.best_params_
print(best_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_estimators': 300, 'min_samples_leaf': 4, 'max_features': 'sqrt'}


In [262]:
base_rf2 = RandomForestClassifier(n_estimators=300, min_samples_leaf=4, max_features='sqrt')
base_rf2.fit(x_train, y_train)
preds2 = base_rf2.predict(x_test)
accuracy_score(y_test,preds2)

0.6136620762298599

In [263]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
cross_validate(base_rf2, x, y, cv=5)

{'fit_time': array([62.79978323, 62.32137895, 61.97563887, 61.15005112, 55.07576919]),
 'score_time': array([2.61142993, 2.49700284, 2.29820085, 2.21649098, 2.26230502]),
 'test_score': array([0.45065148, 0.47218538, 0.42025593, 0.43742197, 0.46535581])}

In [265]:
n = cross_val_score(base_rf2, x, y, cv=5)
n.mean() , n.std()

(0.4478788997173989, 0.019694127365534504)

In [267]:
cross_validate(lg, x, y, cv=5)

{'fit_time': array([45.38520217, 45.38407397, 45.20780516, 39.82966089, 41.19595599]),
 'score_time': array([0.06803012, 0.02923179, 0.02713203, 0.02321196, 0.0264771 ]),
 'test_score': array([0.37052352, 0.39010689, 0.33801498, 0.37578027, 0.44101124])}

In [268]:
n = cross_val_score(lg, x, y, cv=5)
n.mean() , n.std()

(0.38308738092364647, 0.03360717782359277)

### 1) Были использованны признаки количества продукции и ее цены,так как время доставки может зависеть от большего или меньшего количества продуктов и время доставки может быть больше при доставке более дорогих продуктов,был использован номер телефона поставщика,так как время доставки может зависеть от определенного поставщика,так же время доставки может зависеть от выбора метода оплаты,может при оплате оффлайн доставка идет дольше,так же был использован признак названия продуктов,может определнные продукты доставляются дольше,так же признаком был выбран адресс поставщика,ведь время доставки зависит от того,откуда идет доставка,так же был использован признак операционной системы,может с определенных операционных систем запросы обрабатываются дольше.

### 2) Данные были проверены на пропуски для работы модели,так же численные признаки были нормализованны для улучшения предсказательной способности модели,качественные признаки были закодированны для использования в предсказании моделью

### 3) Была выбрана модель случайного леса,так как она больше подходит для задач регрессии и имеет больше возможностей для модификации.Так же тестировалась модель линейной регрессии,так как для числовых признаков она может показывать хорошие результаты 

### 4) На кросс-валидации,после тюнинга,лучше показала себя модель случайного леса